# Word Document to Graph
- using neo4j knowledge graph


In [2]:
!pip install Spire.Doc


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/27.0 MB ? eta -:--:--
    --------------------------------------- 0.3/27.0 MB 10.2 MB/s eta 0:00:03
   - -------------------------------------- 0.9/27.0 MB 11.2 MB/s eta 0:00:03
   -- ------------------------------------- 1.4/27.0 MB 11.3 MB/s eta 0:00:03
   -- ------------------------------------- 2.0/27.0 MB 11.4 MB/s eta 0:00:03
   --- ------------------------------------ 2.5/27.0 MB 11.5 MB/s eta 0:00:03
   ---- ----------------------------------- 3.1/27.0 MB 12.2 MB/s eta 0:00:02
   ----- ---------------------------------- 3.6/27.0 MB 12.0 MB/s eta 0:00:02
   ------ --------------------------------- 4.1/27.0 MB 12.0 MB/s eta 0:00:02
   ------ --------------------------------- 4.7/27.0 MB 12.0 MB/s eta 0:00:02
   ------- -------------------------------- 5.2/27.0 MB 11.9 MB/s eta 0:00:02
   -------- ------------------------------- 5.8/27.0 MB 11.9 MB/s eta 0:00:02
   -------- ------------------------------- 6.0/27.0 MB 11.3 MB/s eta 0

In [2]:
# create function to extract text from a folder of .docx files
import os
from spire.doc import Document

# Function Definition
def extract_text_from_folder(folder_path):
    """
    Extracts text from all .docx files in a folder and stores them in a list.
    Removes first line of text from each document to remove warning or metadata lines
    from Spire.Doc that is not part of the actual document content.

    Args:
    folder_path (str): The path to the folder containing the .docx files.

    Returns:
    list: A list containing the extracted text from each document.
    """
    documents_text = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".docx"):
            filepath = os.path.join(folder_path, filename)
        try:
            document = Document()
            document.LoadFromFile(filepath)
            # Extract text and remove the first line (if it exists)
            text = document.GetText()
            lines = text.splitlines(keepends=True)  # Split by lines, keeping newlines
            if lines:  # Check if there are any lines
                text_without_first_line = "".join(lines[1:])  # Join lines from index 1 onwards
            else:
                text_without_first_line = ""  # Empty string if no lines

            documents_text.append(text_without_first_line)

        except Exception as e:
            print(f"Error processing file {filename}: {e}")
    return documents_text

In [3]:
# Call function on "Documents" folder
folder_path = r"C:\Users\Z4N3R\OneDrive\Desktop\Documents\GitHub\RAGTAGbot\documents"
documents_text = extract_text_from_folder(folder_path)

### Generate Main Body Text
- will be done for each document
- invoking llm

pw = "dj-FX177bkCtDUcAqKXY9obqxkbhU-5SQ7ZoENa3GT0"
uri = "neo4j+s://e143a661.databases.neo4j.io"
uname = "neo4j"

In [15]:
# install langchain_community and langchain_experimental
!pip install langchain-community
!pip install langchain-experimental



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/202.7 kB ? eta -:--:--
   ------------------------------------- - 194.6/202.7 kB 11.5 MB/s eta 0:00:01
   ---------------------------------------- 202.7/202.7 kB 4.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# install dotenv
!pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install neo4j

   ---------------------------------------- 0.0/293.5 kB ? eta -:--:--
   ---------------------------------------- 293.5/293.5 kB 6.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import re
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

load_dotenv()

neo4j_uri = os.getenv("NEO4J_URI")
neo4j_username = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")

os.environ["NEO4J_URI"] = neo4j_uri
os.environ["NEO4J_USERNAME"] = neo4j_username
os.environ["NEO4J_PASSWORD"] = neo4j_password
    
graph = Neo4jGraph()

llm = model

llm_transformer = LLMGraphTransformer(llm=llm)



: 

In [ ]:
def generate_body_text(text, llm):
    prompt = f"""
    You will be given text extracted from a news article PDF.
    The text will contain the main body content, as well as irrelevant sections such as headers, trending news headlines, random metadata, slogans, or the news outlet name. 
    Your task is to extract only the relevant body text from the article, excluding all other irrelevant information.
    To guide you, the body text typically:
    - Is written in paragraph form, with multiple sentences forming a coherent narrative
    - Does not contain short, fragmented phrases or single-sentence headlines
    - Does not include the news outlet name, slogans, or metadata
    - May include quotes or attributions to sources within the paragraphs
    Please output only the extracted body text, without any additional formatting or comments.

    Extracted PDF Text:
    {text}
    """
    response = llm.invoke(prompt)
    
    print(response.content)
    
    clean_text = re.sub(r'\n', '', response.content)
    return clean_text